# Latin Hyper Cube Sampling

In [1]:
import numpy as np
import pandas as pd
import math


In [2]:
def lhs(n_samples, bounds=None, seed=None):
    np.random.seed(seed)
    n_dim = len(bounds)
    result = np.empty((n_samples, n_dim), dtype=int)

    for i in range(n_dim):
        low, high = bounds[i]
        num_values = high - low + 1

        if num_values < n_samples:
            raise ValueError(
                f"Dimension {i}: Only {num_values} unique integer values available "
                f"for {n_samples} requested samples. Increase the range or reduce the sample size."
            )

        values = np.random.choice(np.arange(low, high + 1), size=n_samples, replace=False)
        np.random.shuffle(values)
        result[:, i] = values
    return result

In [3]:
# parameter ranges
potential1_range = [-2000, 2000]  
potential2_range = [-2000, 400]  
potential3_range = [900]  
hold1_range = [0, 1800]  
hold2_range = [0]  
sweep_speed_range = [10, 1000]  
cycle_range = [100]

In [4]:
bounds = [
    potential1_range,
    potential2_range,
    hold1_range,
    sweep_speed_range,
]


samples = lhs(n_samples=30, bounds=bounds, seed=42)
samples

array([[-1197,  -693,   926,   119],
       [ -501, -1401,   653,   419],
       [ -544,   365,   741,   574],
       [ -956, -1719,   141,   955],
       [-1804, -1761,  1618,   821],
       [  554,  -807,   704,   715],
       [  641,  -739,   312,   951],
       [-1789,  -215,   659,    23],
       [ -325,   127,   922,   447],
       [-1445, -1599,   874,   443],
       [  793,  -252,   338,   468],
       [ 1105,  -205,   271,   254],
       [   86,   172,   856,   736],
       [  -76,  -803,   625,    12],
       [-1795,   -25,  1364,   212],
       [ 1142,  -301,  1611,   979],
       [-1679,   213,  1342,   579],
       [ 1905,  -727,   827,    41],
       [-1930,    76,  1051,    86],
       [-1406,   358,  1383,   557],
       [-1529,    63,   353,   387],
       [  990,  -942,  1639,   323],
       [  750, -1290,  1219,   440],
       [ -248,   -15,  1378,   473],
       [  890, -1488,  1394,    14],
       [  711, -1374,   405,   723],
       [ 1483, -1737,  1665,    70],
 

### Create Table

In [5]:
potential1 = samples[:, 0]
potential2 = samples[:, 1]
hold1 = samples[:, 2]
sweep_speed = samples[:, 3]

num_samples = len(potential1)
potential3 = potential3_range * num_samples
hold2 = hold2_range * num_samples
#cycles = [100] * num_samples

interval1 = hold1 + np.abs(potential1 - potential2)/sweep_speed
interval2 = np.abs(potential2 - potential3)/sweep_speed + hold2
cycles = [min(100, math.floor((3600 - interval1[i]) / interval2[i])) for i in range(len(interval1))]
duration = interval1 + cycles*interval2

sample_df = pd.DataFrame({
    'potential1 (mV vs Hg/HgO)': potential1,
    'hold1 (s)': hold1,
    'potential2 (mV vs Hg/HgO)': potential2,
    'hold2 (s)': hold2,
    'potential3 (mV vs Hg/HgO)': potential3,
    'sweep speed (mV/s)': sweep_speed,
    'cycle (P2-P3)': cycles,
    'duration (s)': duration,
    'duration (h)': duration/3600,
    'duration (min)': np.round(duration/60),
})

sample_df


,potential1 (mV vs Hg/HgO),hold1 (s),potential2 (mV vs Hg/HgO),hold2 (s),potential3 (mV vs Hg/HgO),sweep speed (mV/s),cycle (P2-P3),duration (s),duration (h),duration (min)
0,-1197,926,-693,0,900,119,100,2268.890756,0.630247,38.0
1,-501,653,-1401,0,900,419,100,1204.312649,0.334531,20.0
2,-544,741,365,0,900,574,100,835.789199,0.232164,14.0
3,-956,141,-1719,0,900,955,100,416.039791,0.115567,7.0
4,-1804,1618,-1761,0,900,821,100,1942.169306,0.539491,32.0
5,554,704,-807,0,900,715,100,944.644755,0.262401,16.0
6,641,312,-739,0,900,951,100,485.796004,0.134943,8.0
7,-1789,659,-215,0,900,23,59,3587.652174,0.996570,60.0
8,-325,922,127,0,900,447,100,1095.941834,0.304428,18.0
9,-1445,874,-1599,0,900,443,100,1438.455982,0.399571,24.0


### Test LHS

In [6]:
bounds = [
    [0, 10],
    [0, 10],
    [0, 10],
    [0, 10],
]

samples = lhs(n_samples=10, bounds=bounds, seed=42)
samples

array([[ 5,  2,  9, 10],
       [ 8,  5,  3,  0],
       [ 3,  9,  7,  5],
       [ 0,  0,  1,  4],
       [ 7,  6,  5,  1],
       [ 1,  3, 10,  2],
       [ 9, 10,  0,  9],
       [10,  7,  2,  8],
       [ 2,  8,  8,  7],
       [ 4,  4,  4,  3]])